In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import numpy as np
from scipy import special
import pickle
import time
import ase
import quippy
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from mayavi import mlab
from itertools import product, combinations

In [ ]:
rho = 1
L = 7.93701/(rho**(1/3))
d = 1.5/(rho**(1/3))
model = 'soap_classifier_1.h5'
file = 'liquid.xyz'
test_labels = np.full(500,0)

In [ ]:
#function to find list of parameters for this configuration of coordinates
def find_descriptor(coords):    
    #put coordinates into ASE array
    coords_array=np.stack(coords)
    labels=['Ar' for x in range(0,500)]
    my_atoms=ase.Atoms(labels, positions=coords_array)
    #set size of box
    my_atoms.set_cell([L, L, L])
    #Periodic boundary conditions
    my_atoms.pbc=True
    #define descriptor
    desc=quippy.descriptors.Descriptor("soap cutoff={} cutoff_transition_width=0.0 atom_sigma=0.25 n_max =8 l_max=6".format(d))
    descriptors=desc.calc(my_atoms)['data']
    #write parameters to seperate file to be used for machine learning
    #with open('liquid1.8_soap_n=4.txt','ab') as file:
    #    pickle.dump(descriptors,file)
    return descriptors


In [ ]:
#function to create the descriptors for use within the neural network
def create_desc(param):
    D = [sublist for sublist in param]
    test=[]

    for item in D:
        new_g = []
        for k in item:
            new_g.append(np.real(k))
            #new_g.append(np.imag(k))
        test.append(new_g)
    test_data = np.array(test)
    return test_data

In [ ]:
#function to evaluate accuracy and make predictions on test data, and seperate solid and liquid particles for plotting
def make_predictions(desc,coordinates):
    solid_or_liquid = keras.models.load_model(model)
    test_loss, test_acc = solid_or_liquid.evaluate(desc, test_labels, verbose=2)
    predictions = solid_or_liquid.predict(desc)

    solid_coords=[]
    liquid_coords=[]
    for i, value in enumerate(coordinates):
        if np.argmax(predictions[i])==1:
            solid_coords.append(value)
        else:
            liquid_coords.append(value)
    return solid_coords, liquid_coords,test_acc

In [ ]:
#function to plot crystals, solid particles in red and liquid particles in blue, with a cube around it to indicate boundaries
def plot_crystal(solid_coords,liquid_coords):
    fig = plt.figure()
    ax = plt.axes(projection="3d")
    solid_x=[]
    solid_y=[]
    solid_z=[]
    for e,f,g in solid_coords:
        solid_x.append(e)
        solid_y.append(f)
        solid_z.append(g)
    liquid_x=[]
    liquid_y=[]
    liquid_z=[]
    for m,n,o in liquid_coords:
        liquid_x.append(m)
        liquid_y.append(n)
        liquid_z.append(o)
    mlab.points3d([L/2],[L/2],[L/2],mode='cube',scale_factor=L,opacity=0.1,color=(0,0,0))
    mlab.points3d(solid_x,solid_y,solid_z,scale_factor = 1.0, color=(1,0,0),mode='sphere',resolution=12,opacity=1.0)
    mlab.points3d(liquid_x,liquid_y,liquid_z,scale_factor = 1.0, color=(0,0,1),mode='sphere',resolution=12,opacity=1.0)
    mlab.show()
    return

In [ ]:
#function which takes in files and some parameters, creates descriptors for neural network, makes predictions on particles
#and produces colour coded plots. The data can be saved from here to be plotted outside the jupyter notebook using Mayavi
def sol_or_liq(file,L,d):
    start = time.time()
    total_descriptor_time =0
    total_class_time=0
    j=1
    all_coords =[]
    all_desc=[]
    xyz = open(file)
    accuracy=[]
    solid_configs=0
    liquid_configs=0
    a=[]
    #loop over all 101 configurations
    while j<=101:
        atoms = []
        coordinates = []
        try:
            n_atoms = int(xyz.readline())
        except:
            print("End of file reached")
            break
        
        comment = xyz.readline()
        #read through files and puts coordinates into array
        for line in xyz:
            atom,x,y,z = line.split()
            atoms.append(atom)
            coordinates.append(np.array([float(x), float(y), float(z)]))
            if len(coordinates)==n_atoms:
                break
        all_coords.append(coordinates)
        s1 = time.time()
        #create descriptors
        parameters = find_descriptor(coordinates)
        descriptors=create_desc(parameters)
        e1 = time.time()
        t1 = e1-s1    
        total_descriptor_time+=t1
        all_desc.append(descriptors)
        s2=time.time()
        #Evaluate accuracy and make predictions on data
        sc,lc,test_acc = make_predictions(descriptors,coordinates)
        e2=time.time()
        t2=e2-s2
        total_class_time+=t2
        if len(sc)>495:
            solid_configs+=1
        elif len(lc)>495:
            liquid_configs+=1
        else:
            pass
        a.append(len(sc))
        print('\nTest accuracy:', test_acc)
        accuracy.append(test_acc)
        plot_crystal(sc,lc)
        print('Processed configuration {}. Timestamp = {}'.format(j, comment))
        j+=1
    xyz.close()
    end = time.time()
    total_time = end-start
    print('Total time to constructs descriptors = {} s. Total time to make predictions = {} s.'.format(total_descriptor_time,total_class_time))
    print('% (solid) configs classifed accurately = {}'.format((solid_configs/len(accuracy))*100))
    print('% (liquid) configs classifed accurately = {}'.format((liquid_configs/len(accuracy))*100))
    print('Average test accuracy = {}'.format(sum(accuracy)/len(accuracy)*100))
    print('Minimum number of solid-like particles in a config = {}. Maximum = {}'.format(min(a),max(a)))
    return a

In [ ]:
sol_or_liq(file,L,d)